In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.integrate import quad



# Below we set up the functions that calculates the option price, option Delta and option Vega, as well as the final hedging profits. 
# Our calculation shall be vectorized, meaning that the input of stock spot price and stock volatility will be the arrays of all the
# simulated values all at once. This prevents the frequent call for these calculating functions and significantly reduces the running duration of our codes.



# Vectorized stock path simulator, which simulates [n_sims] independent stock paths.
def Heston_path_vectorized(S0, T, r, kappa, theta, xi, rho, v0, dt, n_sims):
    """
    Vectorized Heston stock price.
    The output is a matrix consisting of the stock price at the prescribed cutting points in the time interval [0,T] of all the simulations
    (that is, of shape (n_sims, n_steps = int(T/dt))).
    """
    n_steps = int(T/dt)
    
    dW1 = np.random.normal(0,1,size=(n_sims, n_steps)) * np.sqrt(dt)
    dW2 = rho*dW1 + np.sqrt(1 - rho**2) * np.random.normal(0,1,size=(n_sims, n_steps)) * np.sqrt(dt)
    
    S = np.zeros((n_sims, n_steps+1))
    v = np.zeros((n_sims, n_steps+1))
    S[:,0] = S0
    v[:,0] = v0
    
    for j in range(n_steps):
        vt = np.maximum(v[:,j], 0)
        vt_sqrt = np.sqrt(vt)
        v[:, j+1] = np.maximum(vt + kappa*(theta - vt)*dt + xi*vt_sqrt*dW2[:, j], 0)
        S[:, j+1] = S[:, j] * np.exp((r - 0.5*vt)*dt + vt_sqrt*dW1[:, j])             # These steps follows from the discretenization of the 
                                                                                      #defining equations of Heston's model.
    return S, v


# Vectorized call option price calculator.
def heston_call_price_vec(S0, K, v0, r, t, kappa, theta, xi, rho, phi_max=200, n_phi=2000):
    """
    Vectorized Heston call option price using trapezoidal integration.
    Supports scalar or array input for S0.
    """
    i = 1j
    phi_grid = np.linspace(1e-5, phi_max, n_phi)   # integration grid
    
    # Ensure S0 is array
    S0 = np.atleast_1d(S0)
    v0 = np.atleast_1d(v0)
    log_moneyness = np.log(S0 / K)  # shape (len(S0), n_phi)
    
    def integrand(phi, Pnum):
        u = 0.5 if Pnum == 1 else -0.5
        b = kappa - rho * xi if Pnum == 1 else kappa
        a = kappa * theta

        d = np.sqrt((rho * xi * phi * i - b)**2 - xi**2 * (2 * u * phi * i - phi**2))
        g = (b - rho * xi * phi * i + d) / (b - rho * xi * phi * i - d)

        exp1 = np.exp(i * phi[:, None] * log_moneyness[None, :])  # (n_phi, n_sims)
        C = r * phi[:, None] * i * t + (a / xi**2) * (
            (b - rho * xi * phi[:, None] * i + d[:, None]) * t
            - 2 * np.log((1 - g[:, None] * np.exp(d[:, None] * t)) / (1 - g[:, None]))
        )
        D = (b - rho * xi * phi[:, None] * i + d[:, None]) / xi**2 * (
            (1 - np.exp(d[:, None] * t)) / (1 - g[:, None] * np.exp(d[:, None] * t))
        )

        f = exp1 * np.exp(C + D * v0)
        return np.real(f / (phi[:, None] * i))  # (n_phi, n_sims)

    # Compute integrals
    I1 = np.trapezoid(np.real(integrand(phi_grid, 1)), phi_grid, axis=0)
    I2 = np.trapezoid(np.real(integrand(phi_grid, 2)), phi_grid, axis=0)

    P1 = 0.5 + (1/np.pi)*I1 
    P2 = 0.5 + (1/np.pi)*I2

    # Final call
    call = S0 * P1 - K * np.exp(-r*t) * P2             # This is the semi-closed form of European Heston call price.
    return call


# Vectorized call option Delta calculator.
def heston_call_delta_vec(S0, K, v0, r, t, kappa, theta, xi, rho, phi_max=200, n_phi=2000):
    """
    Vectorized Heston call option Delta using trapezoidal integration.
    Supports scalar or array input for S0.
    """
    i = 1j
    phi = np.linspace(1e-5, phi_max, n_phi)   # integration grid

    # Make sure S0 is vector, K is scalar
    S0 = np.atleast_1d(S0)  # shape (n_sims,)
    v0 = np.atleast_1d(v0)
    log_moneyness = np.log(S0 / K)  # shape (n_sims,)

    def integrand(phi, Pnum):
        u = 0.5 if Pnum == 1 else -0.5
        b = kappa - rho * xi if Pnum == 1 else kappa
        a = kappa * theta

        d = np.sqrt((rho * xi * phi * i - b)**2 - xi**2 * (2 * u * phi * i - phi**2))
        g = (b - rho * xi * phi * i + d) / (b - rho * xi * phi * i - d)

        exp1 = np.exp(i * phi[:, None] * log_moneyness[None, :])  # (n_phi, n_sims)
        C = r * phi[:, None] * i * t + (a / xi**2) * (
            (b - rho * xi * phi[:, None] * i + d[:, None]) * t
            - 2 * np.log((1 - g[:, None] * np.exp(d[:, None] * t)) / (1 - g[:, None]))
        )
        D = (b - rho * xi * phi[:, None] * i + d[:, None]) / xi**2 * (
            (1 - np.exp(d[:, None] * t)) / (1 - g[:, None] * np.exp(d[:, None] * t))
        )

        f = exp1 * np.exp(C + D * v0)
        return np.real(f / (phi[:, None] * i))  # (n_phi, n_sims)

    def integrand_1(phi, Pnum):
        u = 0.5 if Pnum == 1 else -0.5
        b = kappa - rho * xi if Pnum == 1 else kappa
        a = kappa * theta

        d = np.sqrt((rho * xi * phi * i - b)**2 - xi**2 * (2 * u * phi * i - phi**2))
        g = (b - rho * xi * phi * i + d) / (b - rho * xi * phi * i - d)

        exp1 = np.exp(i * phi[:, None] * log_moneyness[None, :])  # (n_phi, n_sims)
        C = r * phi[:, None] * i * t + (a / xi**2) * (
            (b - rho * xi * phi[:, None] * i + d[:, None]) * t
            - 2 * np.log((1 - g[:, None] * np.exp(d[:, None] * t)) / (1 - g[:, None]))
        )
        D = (b - rho * xi * phi[:, None] * i + d[:, None]) / xi**2 * (
            (1 - np.exp(d[:, None] * t)) / (1 - g[:, None] * np.exp(d[:, None] * t))
        )

        f = exp1 * np.exp(C + D * v0)
        return np.real(f)  # (n_phi, n_sims)

    # Integrals along phi-axis
    I1 = np.trapezoid(integrand(phi, 1), phi, axis=0)  # (n_sims,)
    I2 = np.trapezoid(integrand(phi, 2), phi, axis=0)  # (n_sims,)

    J1 = np.trapezoid(integrand_1(phi, 1), phi, axis=0)  # (n_sims,)
    J2 = np.trapezoid(integrand_1(phi, 2), phi, axis=0)  # (n_sims,)

    P1 = 0.5 + (1 / np.pi) * I1

    delta = P1 + (1 / np.pi) * (J1 - J2)  # shape (n_sims,)          # This is the semi-closed form of European Heston call price.

    return delta if delta.shape[0] > 1 else delta.item()


# Vectorized call option Vega calculator.
def heston_call_vega_vec(S0, K, v0, r, t, kappa, theta, xi, rho, phi_max=200, n_phi=2000):
    """
    Vectorized Heston call option Vega using trapezoidal integration.
    Supports scalar or array input for S0.
    """
    i = 1j
    phi = np.linspace(1e-5, phi_max, n_phi)   # integration grid

    # Make sure S0 is vector, K is scalar
    S0 = np.atleast_1d(S0)  # shape (n_sims,)
    v0 = np.atleast_1d(v0)
    log_moneyness = np.log(S0 / K)  # shape (n_sims,)

    def integrand(phi, Pnum):
        u = 0.5 if Pnum == 1 else -0.5
        b = kappa - rho * xi if Pnum == 1 else kappa
        a = kappa * theta

        d = np.sqrt((rho * xi * phi * i - b)**2 - xi**2 * (2 * u * phi * i - phi**2))
        g = (b - rho * xi * phi * i + d) / (b - rho * xi * phi * i - d)

        exp1 = np.exp(i * phi[:, None] * log_moneyness[None, :])  # (n_phi, n_sims)
        C = r * phi[:, None] * i * t + (a / xi**2) * (
            (b - rho * xi * phi[:, None] * i + d[:, None]) * t
            - 2 * np.log((1 - g[:, None] * np.exp(d[:, None] * t)) / (1 - g[:, None]))
        )
        D = (b - rho * xi * phi[:, None] * i + d[:, None]) / xi**2 * (
            (1 - np.exp(d[:, None] * t)) / (1 - g[:, None] * np.exp(d[:, None] * t))
        )

        f = exp1 * np.exp(C + D * v0)
        return np.real(D*f / (phi[:, None] * i))  # (n_phi, n_sims)

    # Integrals along phi-axis
    I1 = np.trapezoid(integrand(phi, 1), phi, axis=0)  # (n_sims,)
    I2 = np.trapezoid(integrand(phi, 2), phi, axis=0)  # (n_sims,)

    vega = S0 * I1 - K * np.exp(-r*t) * I2  # (nS,)        # This is the semi-closed form of European Heston call price.
    return vega
    






# Delta-hedge PnL
def D_hedge_profit_vectorized(S_path, K, T, r, kappa, theta, xi, rho, v_path, dt, n_hedges):
    """
    For Delta hedging, we only need to hold a varying number of the underlying stock other than the option to achieve Delta neutrality.
    """
    
    n_sims = S_path.shape[0]
    scale = int(S_path.shape[1] / n_hedges)
    
    t_b = np.arange(n_hedges-1) * scale        # We set up n_hedges hedging time points with equal gaps in the interval [0,T].
    t_f = np.arange(1, n_hedges) * scale
    
    S_b = S_path[:, t_b]
    S_f = S_path[:, t_f]
    v_b = v_path[:, t_b]
    
    discount_b = np.exp(-r * t_b * dt)
    discount_f = np.exp(-r * t_f * dt)
    
    hedge_step_profit = np.zeros((n_sims, n_hedges-1))
    
    for i in range(n_hedges-1):
        T_remaining = T - t_b[i] * dt
        
        delta_i = heston_call_delta_vec(S_b[:,i], K, v_b[:,i], r, T_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        hedge_step_profit[:, i] = (discount_f[i]*S_f[:, i] - discount_b[i]*S_b[:, i]) * (-delta_i)    # This gives the hedging revenue between the i-th and (i+1)-th hedging points, with discount.
        
    return hedge_step_profit      



# Vega-hedge PnL
def V_hedge_profit_vectorized(S_path, K, K_1, T, T_1, r, kappa, theta, xi, rho, v_path, dt, n_hedges):
    """
    For Vega hedging, we need to hold a varying number of another European option on the same underlying stock but have later maturities
    other than the option to achieve Vega neutrality.
    """
    n_sims = S_path.shape[0]
    scale = int(S_path.shape[1] / n_hedges)
    
    t_b = np.arange(n_hedges-1) * scale
    t_f = np.arange(1, n_hedges) * scale
    
    S_b = S_path[:, t_b]
    S_f = S_path[:, t_f]
    v_b = v_path[:, t_b]
    v_f = v_path[:, t_f]
    
    discount_b = np.exp(-r * t_b * dt)
    discount_f = np.exp(-r * t_f * dt)
    
    hedge_step_profit = np.zeros((n_sims, n_hedges-1))
    
    for i in range(n_hedges-1):
        T_remaining = T - t_b[i] * dt
        T_1_remaining = T_1 - t_b[i] * dt
        
        C_vega_i = heston_call_vega_vec(S_b[:,i], K, v_b[:,i], r, T_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        C1_vega_i = heston_call_vega_vec(S_b[:,i], K_1, v_b[:,i], r, T_1_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        C1_vega_i = np.where(C1_vega_i != 0, C1_vega_i, 0.01)
        
        price_f_i = heston_call_price_vec(S_f[:,i], K_1, v_f[:,i], r, T_1_remaining, kappa, theta, xi, rho, phi_max=200, n_phi=2000)
        price_b_i = heston_call_price_vec(S_b[:,i], K_1, v_b[:,i], r, T_1_remaining, kappa, theta, xi, rho, phi_max=200, n_phi=2000)
        
        hedge_step_profit[:, i] = (discount_f[i]*price_f_i - discount_b[i]*price_b_i) * (-C_vega_i/C1_vega_i)   # This gives the hedging revenue between the i-th and (i+1)-th hedging points, with discount.

    return hedge_step_profit



# Delta-Vega-hedge PnL
def DV_hedge_profit_vectorized(S_path, K, K_1, K_2, T, T_1, T_2, r, kappa, theta, xi, rho, v_path, dt, n_hedges):
    """
    For Delta-Vega hedging, we need to hold varying numbers of two other European options on the same underlying stock but have later maturities
    other than the option to achieve Delta and Vega neutrality.
    """
    n_sims = S_path.shape[0]
    scale = int(S_path.shape[1] / n_hedges)
    
    t_b = np.arange(n_hedges-1) * scale
    t_f = np.arange(1, n_hedges) * scale
    
    S_b = S_path[:, t_b]
    S_f = S_path[:, t_f]
    v_b = v_path[:, t_b]
    v_f = v_path[:, t_f]
    
    discount_b = np.exp(-r * t_b * dt)
    discount_f = np.exp(-r * t_f * dt)
    
    hedge_step_profit = np.zeros((n_sims, n_hedges-1))
    
    for i in range(n_hedges-1):
        T_remaining = T - t_b[i] * dt
        T_1_remaining = T_1 - t_b[i] * dt
        T_2_remaining = T_2 - t_b[i] * dt
        
        vega_i = heston_call_vega_vec(S_b[:,i], K, v_b[:,i], r, T_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        vega1_i = heston_call_vega_vec(S_b[:,i], K_1, v_b[:,i], r, T_1_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        vega2_i = heston_call_vega_vec(S_b[:,i], K_2, v_b[:,i], r, T_2_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        vega2_i = np.where(vega2_i != 0, vega2_i, 0.01)

        delta_i = heston_call_delta_vec(S_b[:,i], K, v_b[:,i], r, T_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        delta1_i = heston_call_delta_vec(S_b[:,i], K_1, v_b[:,i], r, T_1_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        delta2_i = heston_call_delta_vec(S_b[:,i], K_2, v_b[:,i], r, T_2_remaining, kappa, theta, xi, rho,
                          phi_max=200, n_phi=2000)
        
        price1_f_i = heston_call_price_vec(S_f[:,i], K_1, v_f[:,i], r, T_1_remaining, kappa, theta, xi, rho, phi_max=200, n_phi=2000)
        price1_b_i = heston_call_price_vec(S_b[:,i], K_1, v_b[:,i], r, T_1_remaining, kappa, theta, xi, rho, phi_max=200, n_phi=2000)

        price2_f_i = heston_call_price_vec(S_f[:,i], K_2, v_f[:,i], r, T_2_remaining, kappa, theta, xi, rho, phi_max=200, n_phi=2000)
        price2_b_i = heston_call_price_vec(S_b[:,i], K_2, v_b[:,i], r, T_2_remaining, kappa, theta, xi, rho, phi_max=200, n_phi=2000)

        deno_i = delta1_i - delta2_i*vega1_i/vega2_i
        deno_i = np.where(deno_i != 0, deno_i, 0.01)

        alpha_i = (vega_i*delta2_i/vega2_i - delta_i)/deno_i
        beta_i = -(vega_i + alpha_i*vega1_i)/vega2_i
        
        hedge_step_profit[:, i] = (discount_f[i]*price1_f_i - discount_b[i]*price1_b_i) * alpha_i \
        + (discount_f[i]*price2_f_i - discount_b[i]*price2_b_i) * beta_i    # This gives the hedging revenue between the i-th and (i+1)-th hedging points, with discount.
    
    return hedge_step_profit


# Delta Hedged Portfolio profits
def delta_hedged_portfolio_profits_vectorized(S0, K, T, r, kappa, theta, xi, rho, v0, dt, n_sims, n_hedges):  
    """
    Calculate the total profit of a Delta hedging portfolio.
    """
    S_path, v_path = Heston_path_vectorized(S0, T, r, kappa, theta, xi, rho, v0, dt, n_sims)
    hedge_step_profit = D_hedge_profit_vectorized(S_path, K, T, r, kappa, theta, xi, rho, v_path, dt, n_hedges)  
    hedge_total_profit = np.sum(hedge_step_profit, axis=1)    # Adding up all the hedging profits.
    payoff = np.maximum(S_path[:, -1] - K, 0)                     #option payoff.
    portfolio_profit = hedge_total_profit + payoff * np.exp(-r*T)    # Total profits = hedging profits + option payoff.
    return portfolio_profit


# Vega Hedged Portfolio profits
def vega_hedged_portfolio_profits_vectorized(S0, K, K_1, T, T_1, r, kappa, theta, xi, rho, v0, dt, n_sims, n_hedges):
    """
    Calculate the total profit of a Vega hedging portfolio.
    """
    S_path, v_path = Heston_path_vectorized(S0, T, r, kappa, theta, xi, rho, v0, dt, n_sims)
    hedge_step_profit = V_hedge_profit_vectorized(S_path, K, K_1, T, T_1, r, kappa, theta, xi, rho, v_path, dt, n_hedges)
    hedge_total_profit = np.sum(hedge_step_profit, axis=1)
    payoff = np.maximum(S_path[:, -1] - K, 0)
    portfolio_profit = hedge_total_profit + payoff * np.exp(-r*T)
    return portfolio_profit


# Delta-Vega Hedged Portfolio profits
def delta_vega_hedged_portfolio_profits_vectorized(S0, K, K_1, K_2, T, T_1, T_2, r, kappa, theta, xi, rho, v0, dt, n_sims, n_hedges):
    """
    Calculate the total profit of a Delta-Vega hedging portfolio.
    """
    S_path, v_path = Heston_path_vectorized(S0, T, r, kappa, theta, xi, rho, v0, dt, n_sims)
    hedge_step_profit = DV_hedge_profit_vectorized(S_path, K, K_1, K_2, T, T_1, T_2, r, kappa, theta, xi, rho, v_path, dt, n_hedges)
    hedge_total_profit = np.sum(hedge_step_profit, axis=1)
    payoff = np.maximum(S_path[:, -1] - K, 0)
    portfolio_profit = hedge_total_profit + payoff * np.exp(-r*T)
    return portfolio_profit


